In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
#### upload the modify CSV file to your drive (add features name)
os.chdir('/content/gdrive/MyDrive') # change path if needed 

Mounted at /content/gdrive


In [2]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import string
import re
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import time
from nltk.stem import WordNetLemmatizer
from sklearn.utils import shuffle
from nltk import pos_tag
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import Normalizer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from nltk.stem import PorterStemmer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
import numpy as np
import pandas as pd

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
train_data = pd.read_csv('train.csv')
final_test = pd.read_csv('test.csv')
train_data = shuffle(train_data)

In [4]:
## Show vectorizer size without any feature construction 
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    return ' '.join(tokens)
train_data['raw_body'] = train_data['body'].apply(preprocess_text)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(train_data['raw_body'])
X.size

53906

In [5]:
# Load stop words
stop_words = set(stopwords.words('english'))
# Initialize lemmatizer and stemmer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

more_stop_words = [
        "docs",
        "google",
        "just",
        "think",
        "https",
        "org",
        "www",
        "don’t",
        "like",
        "need",
        "it",
        "you’re",
        "use",
        "reddit",
        "thing",
        "I’m",
        "things",
        "good",
        "blog"
        "really",
        "want",
        "maybe",
        "imgur",
        "com",
        "don",
        "actually",
        "that",
        "make",
        "lot",
        "different",
        "doing",
        "that",
        "better",
        "going",
        "great",
        "fo",
        "http",
        "ft",
        "io",
        "page",
        "archive",
        "web",
        "ads",
        "link",
        "ad",
        "proxy",
        "using",
        "hi",
        "subreddit",
        "op",
        "huh",
        "would",
        "html",
        "cache",
        "search",
        "site",
        "lol",
        "could",
        "get",
        "went",
        "used",
        "say",
        "whenever",
        "kind",
        "hello",
        "bot",
        "outline",
        "free",
        "pdf",
        "bit",
        "sure",
        "made",
        "make",
        "something",
        "long",
        "name",
        "take",
        "whatever",
        "however",
        "likely",
        "co",
        "hey",
        "okey",
        "news",
        "bullshit", 
        "crap",
        "newest",
        "hvtargid"
    ]
for letter in string.ascii_lowercase:
    more_stop_words.append(letter)

tokenizer = RegexpTokenizer(r'\w+')
# Define a function to remove stop words, punctuations

def clean_text(text):
    discord_pattern = r"(?:https?://)?discord(?:\.gg|(?:app)?\.com\/invite)\/(\w+)"
    text = re.sub(r'https?://(?:www\.)?youtube(?:-nocookie)?\.com/(?:[^/\n\s]+/)?(?:watch|embed)(?:\.php|/|\?)\S+', '', text)
    text = re.sub(r'https://docs.google.com/viewer\?url=.+', '', text)
    text = re.sub(discord_pattern, '', text)
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'/u/\w+!', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'/r/', '', text)
    text = re.sub(r'/sp', '', text)
    text = re.sub(r"'s", '', text)
    text = re.sub(r'_[a-zA-Z0-9]_[a-zA-Z0-9]+', '', text)
    text = re.sub(r'_', '', text)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    chars_to_remove = stop_words
    chars_to_remove.update(more_stop_words)
    tokens = [token for token in tokens if token not in chars_to_remove]
    filtered_tokens = [token for token in tokens if len(token) > 2]
    return ' '.join(filtered_tokens)

In [6]:
## Bernoulli Naïve Bayes from scratch
class myBernoulliNB:
    def __init__(self, alpha=1.0, binarize=0.0):
        self.alpha = alpha
        self.binarize = binarize

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)
        self.class_count_ = np.zeros(n_classes, dtype=int)
        self.feature_count_ = np.zeros((n_classes, n_features), dtype=float)
        self.class_log_prior_ = np.zeros(n_classes, dtype=float)
        self.alpha *= n_classes

        # Compute class count and feature count
        for i, c in enumerate(self.classes_):
            X_c = X[y == c]
            self.class_count_[i] = X_c.shape[0]
            self.feature_count_[i] = (X_c > self.binarize).sum(axis=0)

        # Compute class log prior
        n_samples = y.shape[0]
        self.class_log_prior_ = np.log(self.class_count_ + self.alpha) - np.log(n_samples + self.alpha * n_classes)

        # Compute feature log probability
        self.feature_log_prob_ = (np.log(self.feature_count_ + self.alpha)
                                  - np.log(self.class_count_[:, np.newaxis] + self.alpha * 2))

    def predict(self, X):
        X = X.toarray()
        # Compute log likelihoods
        neg_prob = np.log(1 - np.exp(self.feature_log_prob_))
        log_prob = np.dot(X, self.feature_log_prob_.T) + np.dot(1 - X, neg_prob.T) + self.class_log_prior_

        # Return class with highest log likelihood
        y_pred = self.classes_[np.argmax(log_prob, axis=1)]

        return y_pred

    def predict_proba(self, X):
        # Compute log likelihoods
        neg_prob = np.log(1 - np.exp(self.feature_log_prob_))
        log_prob = np.dot(X, self.feature_log_prob_.T) + np.dot(1 - X, neg_prob.T) + self.class_log_prior_

        # Convert log probabilities to probabilities
        prob = np.exp(log_prob - log_prob.max(axis=1)[:, np.newaxis])
        prob /= prob.sum(axis=1)[:, np.newaxis]

        return prob

    def score(self, X, y):
        y_pred = self.predict(X)
        acc = np.mean(y_pred == y)
        return acc

    def get_params(self, deep=True):

        params = {"alpha": self.alpha,
        "binarize": self.binarize}
        return params
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
          setattr(self, parameter, value)
        return self
        

In [7]:
# Apply the clean_text() function to the 'body' column of the train_data dataframe
train_data['clean_text'] = train_data['body'].apply(clean_text)

In [8]:
X = train_data['clean_text']
y = train_data['subreddit'].replace({'Ford': 1, 'Musk': 2, 'Obama': 3, 'Trump':4})

Stop words and URL filtering

In [9]:
text_model_svm = Pipeline([('vect', CountVectorizer()),
                       ('normalize', Normalizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(max_iter=1000, random_state=42))])
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
    'clf-svm__alpha': [0.001, 0.01, 0.1, 1, 10],
    'clf-svm__loss': ['hinge'],
    'clf-svm__penalty': ['l2', 'l1', 'elasticnet'],
    'tfidf__use_idf': (True, False)
}
grid_search = GridSearchCV(text_model_svm, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Best parameters:  {'clf-svm__alpha': 0.001, 'clf-svm__loss': 'hinge', 'clf-svm__penalty': 'l2', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
Best score:  0.9470765345765345


In [10]:
text_model_svm = Pipeline([('vect', CountVectorizer()),
                       ('normalize', Normalizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(max_iter=1000, random_state=42))])
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
    'clf-svm__alpha': [0.001, 0.01, 0.1, 1, 10],
    'clf-svm__loss': ['log_loss'],
    'clf-svm__penalty': ['l2', 'l1', 'elasticnet'],
    'tfidf__use_idf': (True, False)
}
grid_search = GridSearchCV(text_model_svm, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Best parameters:  {'clf-svm__alpha': 0.001, 'clf-svm__loss': 'log_loss', 'clf-svm__penalty': 'l2', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
Best score:  0.9192016317016318


In [11]:
param_grid = {'vect__ngram_range': [(1, 1), (1, 2), (1,3)],
              'tfidf__use_idf': [True, False],
              'clf-svm__C': [0.1, 0.5, 1.0, 2.0, 5.0, 10],
              'clf-svm__kernel': ['rbf']}
text_model_svm2 = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SVC(max_iter=1000, random_state=42))])  
grid_search = GridSearchCV(text_model_svm2, param_grid, cv=5)


grid_search.fit(X, y)
# print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Best parameters:  {'clf-svm__C': 2.0, 'clf-svm__kernel': 'rbf', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}
Best score:  0.8997086247086248


In [12]:
text_model = Pipeline([('vect', CountVectorizer()),
                       ('normalize', Normalizer()),
                       ('tfidf', TfidfTransformer()),
                       ('clf', MultinomialNB())])

# define the parameter grid to search over
param_grid = {'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
    'clf__alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10],
}


# perform the grid search
grid_search = GridSearchCV(text_model, param_grid, cv=5, verbose=1)
grid_search.fit(X, y)

# print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best parameters:  {'clf__alpha': 0.5, 'vect__ngram_range': (1, 3)}
Best score:  0.9233682983682984


In [13]:
# Create a pipeline
text_model = Pipeline([('vect', CountVectorizer()),
                       ('clf', myBernoulliNB())])

# Define the parameter grid to search
param_grid = {'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
              'clf__alpha': [0.001, 0.01, 0.1, 1, 10],
              'clf__binarize': [1]}

grid_search = GridSearchCV(text_model, param_grid, cv=5, verbose=1)
grid_search.fit(X, y)

# Print the best hyperparameters found
print("Best hyperparameters:", grid_search.best_params_)

# Print the accuracy
print("Accuracy:", grid_search.best_score_)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
Best hyperparameters: {'clf__alpha': 0.1, 'clf__binarize': 1, 'vect__ngram_range': (1, 1)}
Accuracy: 0.8913364413364414


Lemmatization 

In [14]:
def clean_text2(text):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    filtered_tokens = [token for token in lemmatized_tokens if len(token) > 3]
    return ' '.join(filtered_tokens)

In [15]:
# Apply the clean_text() function to the 'body' column of the train_data dataframe
train_data['clean_text2'] = train_data['body'].apply(clean_text2)
X = train_data['clean_text2']
y = train_data['subreddit'].replace({'Ford': 1, 'Musk': 2, 'Obama': 3, 'Trump':4})

In [16]:
text_model_svm = Pipeline([('vect', CountVectorizer()),
                       ('normalize', Normalizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(max_iter=1000, random_state=42))])
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
    'clf-svm__alpha': [0.001, 0.01, 0.1, 1, 10],
    'clf-svm__loss': ['hinge'],
    'clf-svm__penalty': ['l2', 'l1', 'elasticnet'],
    'tfidf__use_idf': (True, False)
}
grid_search = GridSearchCV(text_model_svm, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Best parameters:  {'clf-svm__alpha': 0.001, 'clf-svm__loss': 'hinge', 'clf-svm__penalty': 'l2', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
Best score:  0.9387043512043511


In [17]:
text_model_svm = Pipeline([('vect', CountVectorizer()),
                       ('normalize', Normalizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(max_iter=1000, random_state=42))])
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
    'clf-svm__alpha': [0.001, 0.01, 0.1, 1, 10],
    'clf-svm__loss': ['log_loss'],
    'clf-svm__penalty': ['l2', 'l1', 'elasticnet'],
    'tfidf__use_idf': (True, False)
}
grid_search = GridSearchCV(text_model_svm, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Best parameters:  {'clf-svm__alpha': 0.001, 'clf-svm__loss': 'log_loss', 'clf-svm__penalty': 'l2', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
Best score:  0.9066822066822067


In [18]:
param_grid = {'vect__ngram_range': [(1, 1), (1, 2), (1,3)],
              'tfidf__use_idf': [True, False],
              'clf-svm__C': [0.1, 0.5, 1.0, 2.0, 5.0, 10],
              'clf-svm__kernel': ['rbf']}
text_model_svm2 = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SVC(max_iter=1000, random_state=42))])  
grid_search = GridSearchCV(text_model_svm2, param_grid, cv=5)


grid_search.fit(X, y)
# print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Best parameters:  {'clf-svm__C': 2.0, 'clf-svm__kernel': 'rbf', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
Best score:  0.8983391608391609


In [19]:
text_model = Pipeline([('vect', CountVectorizer()),
                       ('normalize', Normalizer()),
                       ('tfidf', TfidfTransformer()),
                       ('clf', MultinomialNB())])

# define the parameter grid to search over
param_grid = {'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
    'clf__alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10],
}


# perform the grid search
grid_search = GridSearchCV(text_model, param_grid, cv=5, verbose=1)
grid_search.fit(X, y)

# print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best parameters:  {'clf__alpha': 0.1, 'vect__ngram_range': (1, 2)}
Best score:  0.9164432789432789


In [20]:
# Create a pipeline
text_model = Pipeline([('vect', CountVectorizer()),
                       ('clf', myBernoulliNB())])

# Define the parameter grid to search
param_grid = {'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
              'clf__alpha': [0.001, 0.01, 0.1, 1, 10],
              'clf__binarize': [1]}

grid_search = GridSearchCV(text_model, param_grid, cv=5, verbose=1)
grid_search.fit(X, y)

# Print the best hyperparameters found
print("Best hyperparameters:", grid_search.best_params_)

# Print the accuracy
print("Accuracy:", grid_search.best_score_)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
Best hyperparameters: {'clf__alpha': 0.01, 'clf__binarize': 1, 'vect__ngram_range': (1, 3)}
Accuracy: 0.8370920745920747


Stemming

In [21]:
def clean_text3(text):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    filtered_tokens = [token for token in stemmed_tokens if len(token) > 3]
    return ' '.join(filtered_tokens)

In [22]:
# Apply the clean_text() function to the 'body' column of the train_data dataframe
train_data['clean_text3'] = train_data['body'].apply(clean_text3)
X = train_data['clean_text3']
y = train_data['subreddit'].replace({'Ford': 1, 'Musk': 2, 'Obama': 3, 'Trump':4})

In [23]:
text_model_svm = Pipeline([('vect', CountVectorizer()),
                       ('normalize', Normalizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(max_iter=1000, random_state=42))])
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
    'clf-svm__alpha': [0.001, 0.01, 0.1, 1, 10],
    'clf-svm__loss': ['hinge'],
    'clf-svm__penalty': ['l2', 'l1', 'elasticnet'],
    'tfidf__use_idf': (True, False)
}
grid_search = GridSearchCV(text_model_svm, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Best parameters:  {'clf-svm__alpha': 0.001, 'clf-svm__loss': 'hinge', 'clf-svm__penalty': 'l2', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
Best score:  0.9373251748251749


In [24]:
text_model_svm = Pipeline([('vect', CountVectorizer()),
                       ('normalize', Normalizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(max_iter=1000, random_state=42))])
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
    'clf-svm__alpha': [0.001, 0.01, 0.1, 1, 10],
    'clf-svm__loss': ['log_loss'],
    'clf-svm__penalty': ['l2', 'l1', 'elasticnet'],
    'tfidf__use_idf': (True, False)
}
grid_search = GridSearchCV(text_model_svm, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Best parameters:  {'clf-svm__alpha': 0.001, 'clf-svm__loss': 'log_loss', 'clf-svm__penalty': 'l2', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
Best score:  0.9094794094794094


In [25]:
param_grid = {'vect__ngram_range': [(1, 1), (1, 2), (1,3)],
              'tfidf__use_idf': [True, False],
              'clf-svm__C': [0.1, 0.5, 1.0, 2.0, 5.0, 10],
              'clf-svm__kernel': ['rbf']}
text_model_svm2 = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SVC(max_iter=1000, random_state=42))])  
grid_search = GridSearchCV(text_model_svm2, param_grid, cv=5)


grid_search.fit(X, y)
# print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Best parameters:  {'clf-svm__C': 2.0, 'clf-svm__kernel': 'rbf', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}
Best score:  0.9052836052836053


In [26]:
text_model = Pipeline([('vect', CountVectorizer()),
                       ('normalize', Normalizer()),
                       ('tfidf', TfidfTransformer()),
                       ('clf', MultinomialNB())])

# define the parameter grid to search over
param_grid = {'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
    'clf__alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10],
}


# perform the grid search
grid_search = GridSearchCV(text_model, param_grid, cv=5, verbose=1)
grid_search.fit(X, y)

# print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best parameters:  {'clf__alpha': 0.1, 'vect__ngram_range': (1, 2)}
Best score:  0.9220376845376844


In [27]:
# Create a pipeline
text_model = Pipeline([('vect', CountVectorizer()),
                       ('clf', myBernoulliNB())])

# Define the parameter grid to search
param_grid = {'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
              'clf__alpha': [0.001, 0.01, 0.1, 1, 10],
              'clf__binarize': [1]}

grid_search = GridSearchCV(text_model, param_grid, cv=5, verbose=1)
grid_search.fit(X, y)

# Print the best hyperparameters found
print("Best hyperparameters:", grid_search.best_params_)

# Print the accuracy
print("Accuracy:", grid_search.best_score_)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
Best hyperparameters: {'clf__alpha': 0.1, 'clf__binarize': 1, 'vect__ngram_range': (1, 2)}
Accuracy: 0.8440462315462316


POS Tagging

In [28]:
def clean_text4(text):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    tagged_tokens = pos_tag(tokens)
    filtered_tokens2 = [token for token, pos in tagged_tokens if pos.startswith('N') or pos.startswith('J') or pos.startswith('V')]
    filtered_tokens = [token for token in filtered_tokens2 if len(token) > 3]
    return ' '.join(filtered_tokens)

In [29]:
# Apply the clean_text() function to the 'body' column of the train_data dataframe
train_data['clean_text4'] = train_data['body'].apply(clean_text4)
X = train_data['clean_text4']
y = train_data['subreddit'].replace({'Ford': 1, 'Musk': 2, 'Obama': 3, 'Trump':4})

In [30]:
text_model_svm = Pipeline([('vect', CountVectorizer()),
                       ('normalize', Normalizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(max_iter=1000, random_state=42))])
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
    'clf-svm__alpha': [0.001, 0.01, 0.1, 1, 10],
    'clf-svm__loss': ['hinge'],
    'clf-svm__penalty': ['l2', 'l1', 'elasticnet'],
    'tfidf__use_idf': (True, False)
}
grid_search = GridSearchCV(text_model_svm, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Best parameters:  {'clf-svm__alpha': 0.001, 'clf-svm__loss': 'hinge', 'clf-svm__penalty': 'l2', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
Best score:  0.941501554001554


In [31]:
text_model_svm = Pipeline([('vect', CountVectorizer()),
                       ('normalize', Normalizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(max_iter=1000, random_state=42))])
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
    'clf-svm__alpha': [0.001, 0.01, 0.1, 1, 10],
    'clf-svm__loss': ['log_loss'],
    'clf-svm__penalty': ['l2', 'l1', 'elasticnet'],
    'tfidf__use_idf': (True, False)
}
grid_search = GridSearchCV(text_model_svm, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Best parameters:  {'clf-svm__alpha': 0.001, 'clf-svm__loss': 'log_loss', 'clf-svm__penalty': 'l2', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
Best score:  0.909450271950272


In [32]:
param_grid = {'vect__ngram_range': [(1, 1), (1, 2), (1,3)],
              'tfidf__use_idf': [True, False],
              'clf-svm__C': [0.1, 0.5, 1.0, 2.0, 5.0, 10],
              'clf-svm__kernel': ['rbf']}
text_model_svm2 = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SVC(max_iter=1000, random_state=42))])  
grid_search = GridSearchCV(text_model_svm2, param_grid, cv=5)


grid_search.fit(X, y)
# print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Best parameters:  {'clf-svm__C': 2.0, 'clf-svm__kernel': 'rbf', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}
Best score:  0.8997280497280495


In [33]:
text_model = Pipeline([('vect', CountVectorizer()),
                       ('normalize', Normalizer()),
                       ('tfidf', TfidfTransformer()),
                       ('clf', MultinomialNB())])

# define the parameter grid to search over
param_grid = {'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
    'clf__alpha': [0.1, 0.5, 1.0, 2.0, 5.0, 10],
}


# perform the grid search
grid_search = GridSearchCV(text_model, param_grid, cv=5, verbose=1)
grid_search.fit(X, y)

# print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best parameters:  {'clf__alpha': 0.5, 'vect__ngram_range': (1, 3)}
Best score:  0.9206196581196581


In [34]:
# Create a pipeline
text_model = Pipeline([('vect', CountVectorizer()),
                       ('clf', myBernoulliNB())])

# Define the parameter grid to search
param_grid = {'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
              'clf__alpha': [0.001, 0.01, 0.1, 1, 10],
              'clf__binarize': [1]}

grid_search = GridSearchCV(text_model, param_grid, cv=5, verbose=1)
grid_search.fit(X, y)

# Print the best hyperparameters found
print("Best hyperparameters:", grid_search.best_params_)

# Print the accuracy
print("Accuracy:", grid_search.best_score_)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
Best hyperparameters: {'clf__alpha': 0.1, 'clf__binarize': 1, 'vect__ngram_range': (1, 1)}
Accuracy: 0.8969502719502719


Final Model and Feature Selection

In [62]:
#Load data again
train_data = pd.read_csv('train.csv')
final_test = pd.read_csv('test.csv')
train_data = shuffle(train_data)


stop words an URL filtering

In [63]:
# Load stop words
stop_words = set(stopwords.words('english'))
# Initialize lemmatizer and stemmer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

more_stop_words = [
        "docs",
        "google",
        "just",
        "think",
        "https",
        "org",
        "www",
        "don’t",
        "like",
        "need",
        "it",
        "you’re",
        "use",
        "reddit",
        "thing",
        "I’m",
        "things",
        "good",
        "blog"
        "really",
        "want",
        "maybe",
        "imgur",
        "com",
        "don",
        "actually",
        "that",
        "make",
        "lot",
        "different",
        "doing",
        "that",
        "better",
        "going",
        "great",
        "fo",
        "http",
        "ft",
        "io",
        "page",
        "archive",
        "web",
        "ads",
        "link",
        "ad",
        "proxy",
        "using",
        "hi",
        "subreddit",
        "op",
        "huh",
        "would",
        "html",
        "cache",
        "search",
        "site",
        "lol",
        "could",
        "get",
        "went",
        "used",
        "say",
        "whenever",
        "kind",
        "hello",
        "bot",
        "outline",
        "free",
        "pdf",
        "bit",
        "sure",
        "made",
        "make",
        "something",
        "long",
        "name",
        "take",
        "whatever",
        "however",
        "likely",
        "co",
        "hey",
        "okey",
        "news",
        "bullshit", 
        "crap",
        "newest",
        "hvtargid"
    ]
for letter in string.ascii_lowercase:
    more_stop_words.append(letter)

tokenizer = RegexpTokenizer(r'\w+')
# Define a function to remove stop words, punctuations

def clean_text(text):
    discord_pattern = r"(?:https?://)?discord(?:\.gg|(?:app)?\.com\/invite)\/(\w+)"
    text = re.sub(r'https?://(?:www\.)?youtube(?:-nocookie)?\.com/(?:[^/\n\s]+/)?(?:watch|embed)(?:\.php|/|\?)\S+', '', text)
    text = re.sub(r'https://docs.google.com/viewer\?url=.+', '', text)
    text = re.sub(discord_pattern, '', text)
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'/u/\w+!', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'/r/', '', text)
    text = re.sub(r'/sp', '', text)
    text = re.sub(r"'s", '', text)
    text = re.sub(r'_[a-zA-Z0-9]_[a-zA-Z0-9]+', '', text)
    text = re.sub(r'_', '', text)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    chars_to_remove = stop_words
    chars_to_remove.update(more_stop_words)
    tokens = [token for token in tokens if token not in chars_to_remove]
    filtered_tokens = [token for token in tokens if len(token) > 3]
    return ' '.join(filtered_tokens)

In [64]:
# Apply the clean_text() function to the 'body' column of the train_data dataframe
train_data['clean_text'] = train_data['body'].apply(clean_text)

In [65]:
for subreddit in ['Obama', 'Musk', 'Trump', 'Ford']:
    unwanted_subs = ['elon','musk', 'donald', 'trump', 'ford', 'joe', 'biden'] if subreddit == 'Obama' else \
                    ['barack', 'obama', 'donald', 'trump', 'ford', 'joe', 'biden'] if subreddit == 'Musk' else \
                    ['barack', 'obama','elon', 'musk', 'ford', 'joe', 'biden'] if subreddit == 'Trump' else \
                    ['barack', 'obama','elon', 'musk', 'donald', 'trump', 'joe', 'biden']
    train_data.loc[train_data['subreddit'] == subreddit, 'clean_text'] = train_data.loc[train_data['subreddit'] == subreddit, 'clean_text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in unwanted_subs]))
X = train_data['clean_text']
y = train_data['subreddit'].replace({'Ford': 1, 'Musk': 2, 'Obama': 3, 'Trump':4})

In [66]:
X = train_data['clean_text']
y = train_data['subreddit'].replace({'Ford': 1, 'Musk': 2, 'Obama': 3, 'Trump':4})

In [67]:
# Create a CountVectorizer object
vectorizer = CountVectorizer()

# Fit the vectorizer to the text data
X_new = vectorizer.fit_transform(X)

# Create a SelectKBest object to select the k best features
selecter = SelectKBest(chi2, k=10)

# Transform the text data to select the k best features
X_new = selecter.fit_transform(X_new, y)

# Get the feature names
feature_names = np.array(list(vectorizer.vocabulary_.keys()))
sorted_indices = np.argsort(list(vectorizer.vocabulary_.values()))
sorted_features = feature_names[sorted_indices]

# Print the 10 most important words
selected_words = pd.DataFrame(data={"words" :sorted_features[selecter.get_support()]})
selected_words["scores"] = selecter.scores_[selecter.get_support()]
df_sorted = selected_words.sort_values(by='scores', ascending=False)
df_sorted

,words,scores
4,obama,2728.855556
8,tesla,1653.618896
0,barack,1010.244444
9,trump,1004.266667
6,please,550.031504
5,paywall,496.155556
2,energy,482.847963
3,full,461.562997
1,behind,444.390367
7,power,436.874835


In [68]:
print("Number of features:", len(feature_names))

Number of features: 7602


In [69]:
text_model = Pipeline([('vect', CountVectorizer()),
                       ('normalize', Normalizer()),
                       ('select', SelectKBest(chi2)),
                       ('tfidf', TfidfTransformer()),
                       ('clf', MultinomialNB())])

# define the parameter grid to search over
param_grid = {
    'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
    'select__k': [1000, 3000, 5000, 5500],
    'clf__alpha': [0.5, 0.8, 1.0, 2.0, 4.0],
    'tfidf__use_idf': (True, False)
}


# perform the grid search
grid_search = GridSearchCV(text_model, param_grid, cv=5, verbose=1)
grid_search.fit(X, y)

# print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
Best parameters:  {'clf__alpha': 0.5, 'select__k': 5500, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
Best score:  0.9345765345765346


In [70]:
text_model_svm = Pipeline([
    ('vect', CountVectorizer()),
    ('normalize', Normalizer()),
    ('select', SelectKBest(chi2)),
    ('tfidf', TfidfTransformer()),
    ('clf-svm', SGDClassifier(max_iter=5000))
])

parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'select__k': [1000, 3000, 5000, 'all'],
    'clf-svm__alpha': [1.5e-3, 1.5e-4, 1.5e-5, 1.5e-2, 1.5e-6, 1.5e-7],
    'clf-svm__loss': ['hinge'],
    'clf-svm__penalty': ['l2', 'l1', 'elasticnet'],
    'tfidf__use_idf': (True, False),
}

grid_search = GridSearchCV(text_model_svm, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Best parameters:  {'clf-svm__alpha': 1.5e-05, 'clf-svm__loss': 'hinge', 'clf-svm__penalty': 'elasticnet', 'select__k': 'all', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 3)}
Best score:  0.9624514374514375


In [71]:
text_model_svm = Pipeline([
    ('vect', CountVectorizer()),
    ('normalize', Normalizer()),
    ('select', SelectKBest(chi2)),
    ('tfidf', TfidfTransformer()),
    ('clf-svm', SGDClassifier(max_iter=5000))
])

parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'select__k': [1000, 3000, 5000, 'all'],
    'clf-svm__alpha': [1.5e-3, 1.5e-4, 1.5e-5, 1.5e-2, 1.5e-6, 1.5e-7],
    'clf-svm__loss': ['log_loss'],
    'clf-svm__penalty': ['l2', 'l1', 'elasticnet'],
    'tfidf__use_idf': (True, False),
}

grid_search = GridSearchCV(text_model_svm, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Best parameters:  {'clf-svm__alpha': 1.5e-06, 'clf-svm__loss': 'log_loss', 'clf-svm__penalty': 'l1', 'select__k': 'all', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}
Best score:  0.9624417249417251


lemmatized

In [72]:
# Load stop words
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer

# Define a function to remove stop words and punctuations


# Load stop words
stop_words = set(stopwords.words('english'))
# Initialize lemmatizer and stemmer
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

more_stop_words = [
        "docs",
        "google",
        "just",
        "think",
        "https",
        "org",
        "www",
        "don’t",
        "like",
        "need",
        "it",
        "you’re",
        "use",
        "reddit",
        "thing",
        "I’m",
        "things",
        "good",
        "blog"
        "really",
        "want",
        "maybe",
        "imgur",
        "com",
        "don",
        "actually",
        "that",
        "make",
        "lot",
        "different",
        "doing",
        "that",
        "better",
        "going",
        "great",
        "fo",
        "http",
        "ft",
        "io",
        "page",
        "archive",
        "web",
        "ads",
        "link",
        "ad",
        "proxy",
        "using",
        "hi",
        "subreddit",
        "op",
        "huh",
        "would",
        "html",
        "cache",
        "search",
        "site",
        "lol",
        "could",
        "get",
        "went",
        "used",
        "say",
        "whenever",
        "kind",
        "hello",
        "bot",
        "outline",
        "free",
        "pdf",
        "bit",
        "sure",
        "made",
        "make",
        "something",
        "long",
        "name",
        "take",
        "whatever",
        "however",
        "likely",
        "co",
        "hey",
        "okey",
        "news",
        "bullshit", 
        "crap",
        "newest",
        "hvtargid"
    ]
for letter in string.ascii_lowercase:
    more_stop_words.append(letter)

tokenizer = RegexpTokenizer(r'\w+')
# Define a function to remove stop words, punctuations

def clean_text(text):
    discord_pattern = r"(?:https?://)?discord(?:\.gg|(?:app)?\.com\/invite)\/(\w+)"
    text = re.sub(r'https?://(?:www\.)?youtube(?:-nocookie)?\.com/(?:[^/\n\s]+/)?(?:watch|embed)(?:\.php|/|\?)\S+', '', text)
    text = re.sub(r'https://docs.google.com/viewer\?url=.+', '', text)
    text = re.sub(discord_pattern, '', text)
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'/u/\w+!', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'/r/', '', text)
    text = re.sub(r'/sp', '', text)
    text = re.sub(r"'s", '', text)
    text = re.sub(r'_[a-zA-Z0-9]_[a-zA-Z0-9]+', '', text)
    text = re.sub(r'_', '', text)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    chars_to_remove = stop_words
    chars_to_remove.update(more_stop_words)
    tokens = [token for token in tokens if token not in chars_to_remove]
    tagged_tokens = pos_tag(tokens)
    #filtered_tokens2 = [token for token, pos in tagged_tokens if pos.startswith('N') or pos.startswith('J') or pos.startswith('V')]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    filtered_tokens = [token for token in lemmatized_tokens if len(token) > 3]
    return ' '.join(filtered_tokens)

In [73]:
#Load data again
train_data = pd.read_csv('train.csv')
final_test = pd.read_csv('test.csv')
train_data = shuffle(train_data)

In [74]:
train_data['clean_text'] = train_data['body'].apply(clean_text)
for subreddit in ['Obama', 'Musk', 'Trump', 'Ford']:
    unwanted_subs = ['elon','musk', 'donald', 'trump', 'ford', 'joe', 'biden'] if subreddit == 'Obama' else \
                    ['barack', 'obama', 'donald', 'trump', 'ford', 'joe', 'biden'] if subreddit == 'Musk' else \
                    ['barack', 'obama','elon', 'musk', 'ford', 'joe', 'biden'] if subreddit == 'Trump' else \
                    ['barack', 'obama','elon', 'musk', 'donald', 'trump', 'joe', 'biden']
    train_data.loc[train_data['subreddit'] == subreddit, 'clean_text'] = train_data.loc[train_data['subreddit'] == subreddit, 'clean_text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in unwanted_subs]))
X = train_data['clean_text']
y = train_data['subreddit'].replace({'Ford': 1, 'Musk': 2, 'Obama': 3, 'Trump':4})

In [75]:
text_model = Pipeline([('vect', CountVectorizer()),
                       ('normalize', Normalizer()),
                       ('select', SelectKBest(chi2)),
                       ('tfidf', TfidfTransformer()),
                       ('clf', MultinomialNB())])

# define the parameter grid to search over
param_grid = {
    'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
    'select__k': [1000, 3000, 5000, 5500],
    'clf__alpha': [0.5, 0.8, 1.0, 2.0, 4.0],
    'tfidf__use_idf': (True, False)
}


# perform the grid search
grid_search = GridSearchCV(text_model, param_grid, cv=5, verbose=1)
grid_search.fit(X, y)

# print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
Best parameters:  {'clf__alpha': 0.5, 'select__k': 5000, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}
Best score:  0.934508547008547


In [76]:
text_model_svm = Pipeline([
    ('vect', CountVectorizer()),
    ('normalize', Normalizer()),
    ('select', SelectKBest(chi2)),
    ('tfidf', TfidfTransformer()),
    ('clf-svm', SGDClassifier(max_iter=5000))
])

parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'select__k': [1000, 3000, 5000, 'all'],
    'clf-svm__alpha': [1.5e-3, 1.5e-4, 1.5e-5, 1.5e-2, 1.5e-6, 1.5e-7],
    'clf-svm__loss': ['hinge'],
    'clf-svm__penalty': ['l2', 'l1', 'elasticnet'],
    'tfidf__use_idf': (True, False),
}

grid_search = GridSearchCV(text_model_svm, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Best parameters:  {'clf-svm__alpha': 0.00015, 'clf-svm__loss': 'hinge', 'clf-svm__penalty': 'elasticnet', 'select__k': 5000, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}
Best score:  0.9637626262626263


In [77]:
text_model_svm = Pipeline([
    ('vect', CountVectorizer()),
    ('normalize', Normalizer()),
    ('select', SelectKBest(chi2)),
    ('tfidf', TfidfTransformer()),
    ('clf-svm', SGDClassifier(max_iter=5000))
])

parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'select__k': [1000, 3000, 5000, 'all'],
    'clf-svm__alpha': [1.5e-3, 1.5e-4, 1.5e-5, 1.5e-2, 1.5e-6, 1.5e-7],
    'clf-svm__loss': ['log_loss'],
    'clf-svm__penalty': ['l2', 'l1', 'elasticnet'],
    'tfidf__use_idf': (True, False),
}

grid_search = GridSearchCV(text_model_svm, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Best parameters:  {'clf-svm__alpha': 1.5e-06, 'clf-svm__loss': 'log_loss', 'clf-svm__penalty': 'l1', 'select__k': 5000, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 3)}
Best score:  0.9665695415695416


Pos tagging

In [78]:
# Load stop words
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer

# Define a function to remove stop words and punctuations


# Load stop words
stop_words = set(stopwords.words('english'))
# Initialize lemmatizer and stemmer
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

more_stop_words = [
        "docs",
        "google",
        "just",
        "think",
        "https",
        "org",
        "www",
        "don’t",
        "like",
        "need",
        "it",
        "you’re",
        "use",
        "reddit",
        "thing",
        "I’m",
        "things",
        "good",
        "blog"
        "really",
        "want",
        "maybe",
        "imgur",
        "com",
        "don",
        "actually",
        "that",
        "make",
        "lot",
        "different",
        "doing",
        "that",
        "better",
        "going",
        "great",
        "fo",
        "http",
        "ft",
        "io",
        "page",
        "archive",
        "web",
        "ads",
        "link",
        "ad",
        "proxy",
        "using",
        "hi",
        "subreddit",
        "op",
        "huh",
        "would",
        "html",
        "cache",
        "search",
        "site",
        "lol",
        "could",
        "get",
        "went",
        "used",
        "say",
        "whenever",
        "kind",
        "hello",
        "bot",
        "outline",
        "free",
        "pdf",
        "bit",
        "sure",
        "made",
        "make",
        "something",
        "long",
        "name",
        "take",
        "whatever",
        "however",
        "likely",
        "co",
        "hey",
        "okey",
        "news",
        "bullshit", 
        "crap",
        "newest",
        "hvtargid"
    ]
for letter in string.ascii_lowercase:
    more_stop_words.append(letter)

tokenizer = RegexpTokenizer(r'\w+')
# Define a function to remove stop words, punctuations

def clean_text(text):
    discord_pattern = r"(?:https?://)?discord(?:\.gg|(?:app)?\.com\/invite)\/(\w+)"
    text = re.sub(r'https?://(?:www\.)?youtube(?:-nocookie)?\.com/(?:[^/\n\s]+/)?(?:watch|embed)(?:\.php|/|\?)\S+', '', text)
    text = re.sub(r'https://docs.google.com/viewer\?url=.+', '', text)
    text = re.sub(discord_pattern, '', text)
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'/u/\w+!', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'/r/', '', text)
    text = re.sub(r'/sp', '', text)
    text = re.sub(r"'s", '', text)
    text = re.sub(r'_[a-zA-Z0-9]_[a-zA-Z0-9]+', '', text)
    text = re.sub(r'_', '', text)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    chars_to_remove = stop_words
    chars_to_remove.update(more_stop_words)
    tokens = [token for token in tokens if token not in chars_to_remove]
    tagged_tokens = pos_tag(tokens)
    filtered_tokens2 = [token for token, pos in tagged_tokens if pos.startswith('N') or pos.startswith('J') or pos.startswith('V')]
    #lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    filtered_tokens = [token for token in filtered_tokens2 if len(token) > 3]
    return ' '.join(filtered_tokens)

In [79]:
#Load data again
train_data = pd.read_csv('train.csv')
final_test = pd.read_csv('test.csv')
train_data = shuffle(train_data)

In [80]:
train_data['clean_text'] = train_data['body'].apply(clean_text)
for subreddit in ['Obama', 'Musk', 'Trump', 'Ford']:
    unwanted_subs = ['elon','musk', 'donald', 'trump', 'ford', 'joe', 'biden'] if subreddit == 'Obama' else \
                    ['barack', 'obama', 'donald', 'trump', 'ford', 'joe', 'biden'] if subreddit == 'Musk' else \
                    ['barack', 'obama','elon', 'musk', 'ford', 'joe', 'biden'] if subreddit == 'Trump' else \
                    ['barack', 'obama','elon', 'musk', 'donald', 'trump', 'joe', 'biden']
    train_data.loc[train_data['subreddit'] == subreddit, 'clean_text'] = train_data.loc[train_data['subreddit'] == subreddit, 'clean_text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in unwanted_subs]))
X = train_data['clean_text']
y = train_data['subreddit'].replace({'Ford': 1, 'Musk': 2, 'Obama': 3, 'Trump':4})

In [81]:
text_model = Pipeline([('vect', CountVectorizer()),
                       ('normalize', Normalizer()),
                       ('select', SelectKBest(chi2)),
                       ('tfidf', TfidfTransformer()),
                       ('clf', MultinomialNB())])

# define the parameter grid to search over
param_grid = {
    'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
    'select__k': [1000, 3000, 5000, 5500],
    'clf__alpha': [0.5, 0.8, 1.0, 2.0, 4.0],
    'tfidf__use_idf': (True, False)
}


# perform the grid search
grid_search = GridSearchCV(text_model, param_grid, cv=5, verbose=1)
grid_search.fit(X, y)

# print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
Best parameters:  {'clf__alpha': 1.0, 'select__k': 3000, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}
Best score:  0.9317210567210568


In [82]:
text_model_svm = Pipeline([
    ('vect', CountVectorizer()),
    ('normalize', Normalizer()),
    ('select', SelectKBest(chi2)),
    ('tfidf', TfidfTransformer()),
    ('clf-svm', SGDClassifier(max_iter=5000))
])

parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'select__k': [1000, 3000, 5000, 'all'],
    'clf-svm__alpha': [1.5e-3, 1.5e-4, 1.5e-5, 1.5e-2, 1.5e-6, 1.5e-7],
    'clf-svm__loss': ['hinge'],
    'clf-svm__penalty': ['l2', 'l1', 'elasticnet'],
    'tfidf__use_idf': (True, False),
}

grid_search = GridSearchCV(text_model_svm, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Best parameters:  {'clf-svm__alpha': 0.00015, 'clf-svm__loss': 'hinge', 'clf-svm__penalty': 'l2', 'select__k': 'all', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}
Best score:  0.9595376845376846


In [83]:
text_model_svm = Pipeline([
    ('vect', CountVectorizer()),
    ('normalize', Normalizer()),
    ('select', SelectKBest(chi2)),
    ('tfidf', TfidfTransformer()),
    ('clf-svm', SGDClassifier(max_iter=5000))
])

parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'select__k': [1000, 3000, 5000, 'all'],
    'clf-svm__alpha': [1.5e-3, 1.5e-4, 1.5e-5, 1.5e-2, 1.5e-6, 1.5e-7],
    'clf-svm__loss': ['log_loss'],
    'clf-svm__penalty': ['l2', 'l1', 'elasticnet'],
    'tfidf__use_idf': (True, False),
}

grid_search = GridSearchCV(text_model_svm, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Best parameters:  {'clf-svm__alpha': 0.00015, 'clf-svm__loss': 'log_loss', 'clf-svm__penalty': 'l2', 'select__k': 'all', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
Best score:  0.9540015540015541


Stemmed

In [84]:
# Load stop words
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer

# Define a function to remove stop words and punctuations


# Load stop words
stop_words = set(stopwords.words('english'))
# Initialize lemmatizer and stemmer
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

more_stop_words = [
        "docs",
        "google",
        "just",
        "think",
        "https",
        "org",
        "www",
        "don’t",
        "like",
        "need",
        "it",
        "you’re",
        "use",
        "reddit",
        "thing",
        "I’m",
        "things",
        "good",
        "blog"
        "really",
        "want",
        "maybe",
        "imgur",
        "com",
        "don",
        "actually",
        "that",
        "make",
        "lot",
        "different",
        "doing",
        "that",
        "better",
        "going",
        "great",
        "fo",
        "http",
        "ft",
        "io",
        "page",
        "archive",
        "web",
        "ads",
        "link",
        "ad",
        "proxy",
        "using",
        "hi",
        "subreddit",
        "op",
        "huh",
        "would",
        "html",
        "cache",
        "search",
        "site",
        "lol",
        "could",
        "get",
        "went",
        "used",
        "say",
        "whenever",
        "kind",
        "hello",
        "bot",
        "outline",
        "free",
        "pdf",
        "bit",
        "sure",
        "made",
        "make",
        "something",
        "long",
        "name",
        "take",
        "whatever",
        "however",
        "likely",
        "co",
        "hey",
        "okey",
        "news",
        "bullshit", 
        "crap",
        "newest",
        "hvtargid"
    ]
for letter in string.ascii_lowercase:
    more_stop_words.append(letter)

tokenizer = RegexpTokenizer(r'\w+')
# Define a function to remove stop words, punctuations

def clean_text(text):
    discord_pattern = r"(?:https?://)?discord(?:\.gg|(?:app)?\.com\/invite)\/(\w+)"
    text = re.sub(r'https?://(?:www\.)?youtube(?:-nocookie)?\.com/(?:[^/\n\s]+/)?(?:watch|embed)(?:\.php|/|\?)\S+', '', text)
    text = re.sub(r'https://docs.google.com/viewer\?url=.+', '', text)
    text = re.sub(discord_pattern, '', text)
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'/u/\w+!', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'/r/', '', text)
    text = re.sub(r'/sp', '', text)
    text = re.sub(r"'s", '', text)
    text = re.sub(r'_[a-zA-Z0-9]_[a-zA-Z0-9]+', '', text)
    text = re.sub(r'_', '', text)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    chars_to_remove = stop_words
    chars_to_remove.update(more_stop_words)
    tokens = [token for token in tokens if token not in chars_to_remove]
    #tagged_tokens = pos_tag(tokens)
    #filtered_tokens2 = [token for token, pos in tagged_tokens if pos.startswith('N') or pos.startswith('J') or pos.startswith('V')]
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    filtered_tokens = [token for token in stemmed_tokens if len(token) > 3]
    return ' '.join(filtered_tokens)

In [85]:
#Load data again
train_data = pd.read_csv('train.csv')
final_test = pd.read_csv('test.csv')
train_data = shuffle(train_data)

In [86]:
train_data['clean_text'] = train_data['body'].apply(clean_text)
for subreddit in ['Obama', 'Musk', 'Trump', 'Ford']:
    unwanted_subs = ['elon','musk', 'donald', 'trump', 'ford', 'joe', 'biden'] if subreddit == 'Obama' else \
                    ['barack', 'obama', 'donald', 'trump', 'ford', 'joe', 'biden'] if subreddit == 'Musk' else \
                    ['barack', 'obama','elon', 'musk', 'ford', 'joe', 'biden'] if subreddit == 'Trump' else \
                    ['barack', 'obama','elon', 'musk', 'donald', 'trump', 'joe', 'biden']
    train_data.loc[train_data['subreddit'] == subreddit, 'clean_text'] = train_data.loc[train_data['subreddit'] == subreddit, 'clean_text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in unwanted_subs]))
X = train_data['clean_text']
y = train_data['subreddit'].replace({'Ford': 1, 'Musk': 2, 'Obama': 3, 'Trump':4})

In [91]:
text_model = Pipeline([('vect', CountVectorizer()),
                       ('normalize', Normalizer()),
                       ('select', SelectKBest(chi2)),
                       ('tfidf', TfidfTransformer()),
                       ('clf', MultinomialNB())])

# define the parameter grid to search over
param_grid = {
    'vect__ngram_range': [(1, 1), (1, 2),(1, 3)],
    'select__k': [1000, 3000, 'all'],
    'clf__alpha': [0.5, 0.8, 1.0, 2.0, 4.0],
    'tfidf__use_idf': (True, False)
}


# perform the grid search
grid_search = GridSearchCV(text_model, param_grid, cv=5, verbose=1)
grid_search.fit(X, y)

# print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Best parameters:  {'clf__alpha': 2.0, 'select__k': 1000, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
Best score:  0.9358780108780109


In [90]:
text_model_svm = Pipeline([
    ('vect', CountVectorizer()),
    ('normalize', Normalizer()),
    ('select', SelectKBest(chi2)),
    ('tfidf', TfidfTransformer()),
    ('clf-svm', SGDClassifier(max_iter=5000))
])

parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'select__k': [1000, 3000, 'all'],
    'clf-svm__alpha': [1.5e-3, 1.5e-4, 1.5e-5, 1.5e-2, 1.5e-6, 1.5e-7],
    'clf-svm__loss': ['hinge'],
    'clf-svm__penalty': ['l2', 'l1', 'elasticnet'],
    'tfidf__use_idf': (True, False),
}

grid_search = GridSearchCV(text_model_svm, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best parameters:  {'clf-svm__alpha': 0.0015, 'clf-svm__loss': 'hinge', 'clf-svm__penalty': 'l2', 'select__k': 3000, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}
Best score:  0.9637432012432013


In [92]:
text_model_svm = Pipeline([
    ('vect', CountVectorizer()),
    ('normalize', Normalizer()),
    ('select', SelectKBest(chi2)),
    ('tfidf', TfidfTransformer()),
    ('clf-svm', SGDClassifier(max_iter=5000))
])

parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'select__k': [1000, 3000, 'all'],
    'clf-svm__alpha': [1.5e-3, 1.5e-4, 1.5e-5, 1.5e-2, 1.5e-6, 1.5e-7],
    'clf-svm__loss': ['log_loss'],
    'clf-svm__penalty': ['l2', 'l1', 'elasticnet'],
    'tfidf__use_idf': (True, False),
}

grid_search = GridSearchCV(text_model_svm, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best parameters:  {'clf-svm__alpha': 1.5e-05, 'clf-svm__loss': 'log_loss', 'clf-svm__penalty': 'l2', 'select__k': 3000, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 3)}
Best score:  0.9707070707070707


Final Model Select

In [93]:
final_test['clean_text'] = final_test['body'].apply(clean_text)

In [94]:
final_X_test = final_test['clean_text']

In [95]:
best_params = grid_search.best_params_
final_test['clean_text'] = final_test['body'].apply(clean_text)
final_X_test = final_test['clean_text']

text_model_svm_final = Pipeline([
    ('vect', CountVectorizer(ngram_range=best_params['vect__ngram_range'])),
    ('tfidf', TfidfTransformer(use_idf=best_params['tfidf__use_idf'])),
    ('normalize', Normalizer()),
    ('select', SelectKBest(k=best_params['select__k'])),
    ('clf-svm', SGDClassifier(alpha=best_params['clf-svm__alpha'], 
                              loss=best_params['clf-svm__loss'], 
                              penalty=best_params['clf-svm__penalty']))
])
text_model_svm_final.fit(X, y)

# Evaluate the final model on the test dataset
y_pred = text_model_svm_final.predict(final_X_test)
y_pred


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2,
       2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 4, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3,
       3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4,
       4, 4, 4, 4, 4, 4, 1, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 1, 4, 3, 4])

In [97]:
# Create a reverse dictionary to map predicted values to original labels
reverse_dict = {1: 'Ford', 2: 'Musk', 3: 'Obama', 4: 'Trump'}

# Map predicted values to original labels
predicted_labels = [reverse_dict[pred] for pred in y_pred]

# Create a new DataFrame with the id and subreddit columns
final_predict2_0306 = pd.DataFrame({'id': range(len(predicted_labels)), 'subreddit': predicted_labels})
final_predict2_0306.to_csv('final_predict666666666666666.csv', index=False)

In [98]:
len(y_pred)

279